In [546]:
from utils import *
from pprint import pprint

In [547]:
house = importfile('hw3_house_votes_84.csv', ',')
housecategory = {}
for i in house[0]:
    housecategory[i] = 'categorical'
housecategory["class"] = 'class'
housedata = np.array(house[1:]).astype(float)

# print(len(housedata))
# print(housecategory)

#class in last column

In [548]:
wine = importfile('hw3_wine.csv', '\t')
winecategory = {}
for i in wine[0]:
    winecategory[i] = 'numerical'
winecategory["# class"] = 'class'
winedata = np.array(wine[1:]).astype(float)
# winedata = np.array([[float(item) for item in onelist] for onelist in wine[1:]])

# print(len(winedata))
# print(winecategory)

#class in first column

In [549]:
cancer = importfile('hw3_cancer.csv', '\t')
cancercategory = {}
for i in cancer[0]:
    cancercategory[i] = 'numerical'
cancercategory["Class"] = 'class'
cancerdata = np.array(cancer[1:]).astype(float)

#print(len(cancerdata))
#print(cancercategory)

#class in last column

In [550]:
cmc = importfile('cmc.data', ',')
cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
"Husband's education":"categorical","Number of children ever born":"numerical",
"Wife's religion":"binary","Wife's now working?":"binary",
"Husband's occupation":"categorical","Standard-of-living index":"categorical",
"Media exposure":"binary","Contraceptive method used":"class"}
cmcdata = np.array(cmc).astype(int)

# print(len(cmcdata))
# print(cmccategory)

#class in last column

In [551]:
w = {list(cmccategory.keys())[0]:cmccategory[list(cmccategory.keys())[0]]}

print(w)

{"Wife's age": 'numerical'}


In [552]:
def id3cat(collist, listattribution):
    original_ent = entropy(collist[-1])
    smallest_ent = 1
    i = 0

    # bestindex = i
    best = listattribution[i]
    for attributes in listattribution[:-1]: # I keep the last column: the target/label.
        liskey = list(Counter(collist[i]).keys())
        listofcategory = []
        for value in liskey:
            index = [idx for idx, element in enumerate(collist[i]) if element == value]
            category = np.array(collist[-1][index]) 
            listofcategory.append(category) # list of nparrays of target/label/categories.

        ent = 0
        for cat in listofcategory:
            a = len(cat)/len(collist[i]) # This is probability
            ent += a * entropy(cat) # probability multiple by entropy

        if ent < smallest_ent:
            smallest_ent = ent
            best = attributes
            # bestindex = i
        i+=1

    return best, original_ent-smallest_ent

In [553]:
def id3bestseperate(dataset, attributes:dict):
    # dataset in is the dataset by row. 
    # attributes is the dictionary of attributes:type 
    # types: numerical, categorical, binary.
    datasetbycolumn = dataset.T
    classindex = list(attributes.values()).index("class")
    originalentrophy = entropy(datasetbycolumn[classindex])
    smallestentrophy = originalentrophy
    print("original entropy is: ", originalentrophy)
    thresholdvalue = -1

    i = 0
    bestattribute = {list(attributes.keys())[i]:attributes[list(attributes.keys())[i]]}
    attributesinuse = list(attributes.keys())[1:] if (classindex == 0) else list(attributes.keys())[:classindex]
    # datasetinuse = datasetbycolumn[1:] if (classindex == 0) else datasetbycolumn[:classindex]

    for attribute in attributesinuse:
        idx = i+1 if classindex == 0 else i

        if attributes[attribute] == "categorical" or attributes[attribute] == "binary":
            print("attribute #",i," is ",attribute)
            listofkeys = list(Counter(datasetbycolumn[idx]).keys())
            listofcategory = [] # this is the list of categorical values.
            
            for key in listofkeys:
                indexlist = [idex for idex, element in enumerate(datasetbycolumn[idx]) if element == key]
                category = np.array(datasetbycolumn[classindex][indexlist])
                listofcategory.append(category)

            entropynow = 0

            for ctgry in listofcategory:
                a = len(ctgry)/len(datasetbycolumn[idx]) # This is probability
                entropynow += a * entropy(ctgry)

            if entropynow < smallestentrophy:
                smallestentrophy = entropynow
                bestattribute = {attribute:attributes[attribute]}
            
        elif attributes[attribute] == "numerical":
            print("attribute #",i," is ",attribute)

            datasetsort = datasetbycolumn.T[datasetbycolumn.T[:,idx].argsort(kind='quicksort')].T
            currentthreshold = (datasetsort[idx][1]+datasetsort[idx][0])/2
            k = 1
            while k < len(datasetsort.T):
                currentthreshold = (datasetsort[idx][k]+datasetsort[idx][k-1])/2
                listofcategory = [datasetsort[classindex][:k],datasetsort[classindex][k:]]
                entropynow = 0

                for ctgry in listofcategory:
                    a = len(ctgry)/len(datasetbycolumn[idx]) # This is probability
                    entropynow += a * entropy(ctgry)

                if entropynow < smallestentrophy:
                    smallestentrophy = entropynow
                    thresholdvalue = currentthreshold
                    bestattribute = {attribute:attributes[attribute]}    
                k += 1
        i += 1

    gain = originalentrophy-smallestentrophy
    # set first attribution dictionary {key:type} to the best attributes.
    return bestattribute, thresholdvalue, gain

In [554]:
# a = winedata.T.copy()
# print(a)
# print(len(a))
# asort = a.T[a.T[:,1].argsort(kind='quicksort')].T
# print(asort[1])
# print((asort[1][1]+asort[1][0])/2)
# print(len(asort.T))

In [555]:
def dropbyindex(data, category, listindex):
    newdata = np.delete(data.T, listindex,0).T
    keytoremove = [list(category.keys())[i] for i in listindex]
    newcategory = category.copy()
    [newcategory.pop(key) for key in keytoremove]
    return newdata, newcategory

In [561]:
removeindex0= [1,2,3,5,6,7,8]
data1,category1=dropbyindex(winedata,winecategory,removeindex0)
print(len(data1))
bestattribute, thresholdvalue, gain = id3bestseperate(data1,category1)
print(bestattribute, thresholdvalue, gain)

178
original entropy is:  1.5668222768551812
attribute # 0  is  4
attribute # 1  is  9
attribute # 2  is  10
attribute # 3  is  11
attribute # 4  is  12
attribute # 5  is  13
{'12': 'numerical'} 2.475 0.6173242577120465


In [557]:
removeindex= [1,2,8,12]
housedata1,housecategory1=dropbyindex(housedata,housecategory,removeindex)
bestattribute, thresholdvalue, gain = id3bestseperate(housedata1,housecategory1)
print(bestattribute)
ba,ga = id3cat(housedata1.T,list(housecategory1.keys()))
ba


original entropy is:  0.9623080486960709
attribute # 0  is  #handicapped-infants
attribute # 1  is  physician-fee-freeze
attribute # 2  is  el-salvador-adi
attribute # 3  is  religious-groups-in-schools
attribute # 4  is  anti-satellite-test-ban
attribute # 5  is  aid-to-nicaraguan-contras
attribute # 6  is  immigration
attribute # 7  is  synfuels-corporation-cutback
attribute # 8  is  education-spending
attribute # 9  is  crime
attribute # 10  is  duty-free-exports
attribute # 11  is  export-administration-act-south-africa
{'physician-fee-freeze': 'categorical'}


'physician-fee-freeze'